In [ ]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.4 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow

# 0) BASE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import mlflow
import mlflow.tensorflow
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.layers import LSTM
import keras.backend as K
from keras.optimizers import Adam
from keras import Sequential
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
import warnings

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ASAC_4기_기업프로젝트_SK하이닉스/1. 데이터 전처리/5. 최종 데이터셋(2024.00.00 기준)/final_df_reordered.csv')
#columns_to_drop = ['TA_tm_before', 'start_dt_tm', 'end_dt_tm', 'TA_tm_after', 'P_Time', 'sequence']
#df.drop(columns_to_drop, axis=1, inplace=True)
df.head()

,sequence,RCP_ID,STEP_NM,TA_tm_before,start_dt_tm,end_dt_tm,TA_tm_after,P_Time,step_hours,last_hours,...,stddev_val_17_Diff,stddev_val_18_Diff,stddev_val_19_Diff,max_val_pca_Diff,min_val_pca_Diff,stddev_val_pca_Diff,mean_val_pca_Diff,range_val_pca_Diff,median_val_pca_Diff,diff_hours
0,0.0,13,36,2022-04-13 10:21:24,2022-04-15 15:37:07,2022-04-15 15:51:34,2022-04-22 16:38:35,2022-04-14 04:00:00,0.240833,53.261944,...,-0.011524,0.219372,0.332226,-48.464392,-48.464392,-48.464392,-48.464392,-48.464392,-48.464392,169.024444
1,1.0,13,36,2022-04-13 10:21:24,2022-04-15 15:50:06,2022-04-15 15:50:42,2022-04-22 16:38:35,2022-04-14 04:00:00,0.010000,53.478333,...,-0.005883,-0.359641,-0.515629,-40.148962,-40.148962,-40.148962,-40.148962,-40.148962,-40.148962,168.808056
2,2.0,1,36,2022-04-13 10:21:24,2022-04-15 15:51:34,2022-04-21 00:30:59,2022-04-22 16:38:35,2022-04-14 04:00:00,128.656944,53.502778,...,-0.014540,-0.178041,-0.189356,-73.647082,-73.647082,-73.647082,-73.647082,-73.647082,-73.647082,168.783611
3,3.0,1,36,2022-04-13 10:21:24,2022-04-15 15:51:46,2022-04-15 15:58:36,2022-04-22 16:38:35,2022-04-14 04:00:00,0.113889,53.506111,...,0.036082,-0.248223,-0.109616,-73.647082,-73.647082,-73.647082,-73.647082,-73.647082,-73.647082,168.780278
4,4.0,1,36,2022-04-13 10:21:24,2022-04-15 15:58:37,2022-04-15 16:12:31,2022-04-22 16:38:35,2022-04-14 04:00:00,0.231667,53.620278,...,-0.072168,0.233206,0.646370,-33.192453,-33.192453,-33.192453,-33.192453,-33.192453,-33.192453,168.666111


In [ ]:
# metric 함수
def weighted_rmse1(y_true, y_pred):
    diff = tf.square(y_true - y_pred)
    rul_t = 1 - (y_true / tf.reduce_max(y_true))
    n_metric = diff * rul_t
    return K.sqrt(K.sum(n_metric) / K.sum(rul_t))

def weighted_rmse2(y_true, y_pred):
    diff = tf.square(y_true - y_pred)
    rul_t = 1 / (y_true + 1)
    weighted_diff = diff * rul_t
    return K.sqrt(K.sum(weighted_diff) / K.sum(rul_t))

# 1. MLflow를 활용한 모델 학습 Tracking
* auto logging을 활용한 모델 학습 기록 관리
* auto logging + custom logging

In [ ]:
# 데이터 전처리 및 Train/Test 분리 : fail_cnt 17 기준으로
def prepare_data(df, fail_cnt, time_steps):
    train_df = df[df['sequence'] <= fail_cnt]
    test_df = df[df['sequence'] > fail_cnt]

    y_train = train_df['diff_hours']
    y_test = test_df['diff_hours']
    x_train = train_df.drop(columns=['diff_hours', 'sequence', 'TA_tm_before', 'start_dt_tm', 'end_dt_tm', 'TA_tm_after', 'P_Time'])
    x_test = test_df.drop(columns=['diff_hours', 'sequence', 'TA_tm_before', 'start_dt_tm', 'end_dt_tm', 'TA_tm_after', 'P_Time'])

    # 데이터 스케일링
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    # 시퀀스 데이터 구성
    x_train_seq = np.array([x_train_scaled[i: i + time_steps] for i in range(len(x_train_scaled) - time_steps)])
    x_test_seq = np.array([x_test_scaled[i: i + time_steps] for i in range(len(x_test_scaled) - time_steps)])
    y_train_seq = y_train[time_steps:]
    y_test_seq = y_test[time_steps:]

    return x_train_seq, x_test_seq, y_train_seq, y_test_seq

In [ ]:
# 시퀀스 길이
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

## 1-1) auto logging

In [ ]:
mlflow.set_tracking_uri("http://3.36.101.198:5000")
mlflow.set_experiment("lstm_experiment_1")

<Experiment: artifact_location='mlflow-artifacts:/709938647007483152', creation_time=1715002290447, experiment_id='709938647007483152', last_update_time=1715002290447, lifecycle_stage='active', name='lstm_experiment_1', tags={}>

In [ ]:
### 1트 ### : 시퀀스 길이 1임ㅋㅋ
# 모델 구성 및 컴파일
mlflow.tensorflow.autolog()

model = Sequential([
    LSTM(128, input_shape=(1, x_train.shape[1],), return_sequences=True),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(1)
])

# 입력 데이터의 차원 변경
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error',
              metrics=['mean_squared_error', 'mean_absolute_error', weighted_rmse1, weighted_rmse2])

# 모델 훈련
with mlflow.start_run(run_name="LSTM_Model_1"):
    model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# 모델 평가
loss, mse, mae, wrmse1, wrmse2 = model.evaluate(x_test, y_test)
print(f'Test MSE: {mse}, Test MAE: {mae}, Weighted RMSE 1: {wrmse1}, Weighted RMSE 2: {wrmse2}')

2024/05/06 15:11:47 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/10
 5/14 [=========>....................] - ETA: 0s - loss: 7585.0615 - mean_squared_error: 7585.0615 - mean_absolute_error: 66.3060 - weighted_rmse1: 52.6701 - weighted_rmse2: 43.4665  

10/14 [====================>.........] - ETA: 0s - loss: 7495.2241 - mean_squared_error: 7495.2241 - mean_absolute_error: 64.8060 - weighted_rmse1: 53.3784 - weighted_rmse2: 42.0507

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


14/14 [==============================] - 17s 826ms/step - loss: 7756.7114 - mean_squared_error: 7756.7114 - mean_absolute_error: 65.2464 - weighted_rmse1: 54.9620 - weighted_rmse2: 42.4716 - val_loss: 4423.5361 - val_mean_squared_error: 4423.5361 - val_mean_absolute_error: 51.0164 - val_weighted_rmse1: 50.4523 - val_weighted_rmse2: 50.5964
Epoch 2/10
14/14 [==============================] - 13s 1s/step - loss: 7712.3525 - mean_squared_error: 7712.3525 - mean_absolute_error: 64.9750 - weighted_rmse1: 56.6903 - weighted_rmse2: 42.1616 - val_loss: 4369.0601 - val_mean_squared_error: 4369.0601 - val_mean_absolute_error: 50.5093 - val_weighted_rmse1: 49.9515 - val_weighted_rmse2: 50.0929
Epoch 3/10
14/14 [==============================] - 8s 641ms/step - loss: 7553.9849 - mean_squared_error: 7553.9849 - mean_absolute_error: 63.9125 - weighted_rmse1: 54.0864 - weighted_rmse2: 41.7137 - val_loss: 4180.1011 - val_mean_squared_error: 4180.1011 - val_mean_absolute_error: 48.7413 - val_weighted_r

2024/05/06 15:14:12 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp74bk2yok/model, flavor: tensorflow). Fall back to return ['tensorflow==2.15.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
2024/05/06 15:14:12 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


1489/1489 [==============================] - 7s 4ms/step - loss: 3559.1765 - mean_squared_error: 3559.1765 - mean_absolute_error: 43.2468 - weighted_rmse1: 42.7957 - weighted_rmse2: 42.8954
Test MSE: 3559.176513671875, Test MAE: 43.246788024902344, Weighted RMSE 1: 42.795650482177734, Weighted RMSE 2: 42.89542007446289


In [ ]:
### 2트 ### : 시퀀스 길이 24
# 모델 구성 및 컴파일
mlflow.tensorflow.autolog()

# 시퀀스 길이 설정
time_steps = 24

x_train, x_test, y_train, y_test = prepare_data(df, 17, time_steps)

model = Sequential([
    LSTM(128, input_shape=(time_steps, x_train.shape[2]), return_sequences=True),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error', 'mean_absolute_error', weighted_rmse1, weighted_rmse2])

# 모델 훈련
with mlflow.start_run(run_name="LSTM_Model_2"):
    model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# 모델 평가
loss, mse, mae, wrmse1, wrmse2 = model.evaluate(x_test, y_test)
print(f'Test MSE: {mse}, Test MAE: {mae}, Weighted RMSE 1: {wrmse1}, Weighted RMSE 2: {wrmse2}')

2024/05/06 15:36:06 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/10
 6/13 [============>.................] - ETA: 0s - loss: 7833.4805 - mean_squared_error: 7833.4805 - mean_absolute_error: 66.6792 - weighted_rmse1: 59.8361 - weighted_rmse2: 44.6808  

11/13 [========================>.....] - ETA: 0s - loss: 7042.2700 - mean_squared_error: 7042.2700 - mean_absolute_error: 60.8413 - weighted_rmse1: 54.4835 - weighted_rmse2: 40.0289

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13/13 [==============================] - 21s 1s/step - loss: 6769.4268 - mean_squared_error: 6769.4268 - mean_absolute_error: 59.4469 - weighted_rmse1: 52.8918 - weighted_rmse2: 39.1388 - val_loss: 4079.7866 - val_mean_squared_error: 4079.7866 - val_mean_absolute_error: 47.8875 - val_weighted_rmse1: 47.4181 - val_weighted_rmse2: 47.5919
Epoch 2/10
13/13 [==============================] - 10s 822ms/step - loss: 6345.2212 - mean_squared_error: 6345.2212 - mean_absolute_error: 55.8260 - weighted_rmse1: 49.9032 - weighted_rmse2: 36.9648 - val_loss: 3794.1177 - val_mean_squared_error: 3794.1177 - val_mean_absolute_error: 45.2706 - val_weighted_rmse1: 44.8376 - val_weighted_rmse2: 44.9937
Epoch 3/10
13/13 [==============================] - 10s 805ms/step - loss: 6123.8018 - mean_squared_error: 6123.8018 - mean_absolute_error: 53.8979 - weighted_rmse1: 49.9993 - weighted_rmse2: 36.3081 - val_loss: 3652.9941 - val_mean_squared_error: 3652.9941 - val_mean_absolute_error: 44.0544 - val_weighted_

2024/05/06 15:38:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpy5u57tqp/model, flavor: tensorflow). Fall back to return ['tensorflow==2.15.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
2024/05/06 15:38:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


1489/1489 [==============================] - 6s 4ms/step - loss: 3338.2075 - mean_squared_error: 3338.2075 - mean_absolute_error: 41.5450 - weighted_rmse1: 41.1824 - weighted_rmse2: 41.3139
Test MSE: 3338.20751953125, Test MAE: 41.544960021972656, Weighted RMSE 1: 41.182403564453125, Weighted RMSE 2: 41.31391525268555


## 1-2) auto logging + custom logging

In [ ]:
## 시퀀스 길이에 따른 모델 훈련 및 로깅 ##

In [ ]:
mlflow.set_tracking_uri("http://3.36.101.198:5000")
mlflow.set_experiment("lstm_experiment_sequence_test")

# 시퀀스 길이를 변화시키면서 모델을 훈련시키고 결과 로깅 : 시퀀스 길이 1부터 50까지
for sequence_length in range(1, 51):
    with mlflow.start_run(run_name=f"LSTM_SEQUENCE_Model_{sequence_length}"):
        x_train, x_test, y_train, y_test = prepare_data(df, 17, sequence_length)

        model = Sequential([
            LSTM(128, input_shape=(sequence_length, x_train.shape[2]), return_sequences=True),
            LSTM(64, return_sequences=True),
            LSTM(32),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error', 'mean_absolute_error', weighted_rmse1, weighted_rmse2])

        # Auto Logging 활성화
        mlflow.tensorflow.autolog()

        # 모델 훈련
        model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

        # 모델 평가
        loss, mse, mae, wrmse1, wrmse2 = model.evaluate(x_test, y_test)
        mlflow.log_metric("Test MSE", mse)
        mlflow.log_metric("Test MAE", mae)
        mlflow.log_metric("Weighted RMSE 1", wrmse1)
        mlflow.log_metric("Weighted RMSE 2", wrmse2)

        print(f'Sequence Length {sequence_length}: Test MSE: {mse}, Test MAE: {mae}, Weighted RMSE 1: {wrmse1}, Weighted RMSE 2: {wrmse2}')


2024/05/07 05:58:34 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/10


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7ec1ac66e440>
Traceback (most recent call last):
  File "/usr/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


 1/14 [=>............................] - ETA: 1:34 - loss: 9319.5332 - mean_squared_error: 9319.5332 - mean_absolute_error: 73.2273 - weighted_rmse1: 67.4044 - weighted_rmse2: 46.0370

14/14 [==============================] - ETA: 0s - loss: 7709.2646 - mean_squared_error: 7709.2646 - mean_absolute_error: 65.0071 - weighted_rmse1: 56.6713 - weighted_rmse2: 42.8392

...

14/14 [==============================] - 5s 357ms/step - loss: 6645.8057 - mean_squared_error: 6645.8057 - mean_absolute_error: 56.4056 - weighted_rmse1: 49.7326 - weighted_rmse2: 37.6879 - val_loss: 3590.6743 - val_mean_squared_error: 3590.6743 - val_mean_absolute_error: 43.5031 - val_weighted_rmse1: 43.0442 - val_weighted_rmse2: 43.1414


2024/05/07 05:59:34 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id adf8496741a04249b7d6b0e5d06f5df9: Failed to log run data: Exception: INVALID_PARAMETER_VALUE: The run adf8496741a04249b7d6b0e5d06f5df9 must be in 'active' lifecycle_stage.


1/1 [==============================] - 1s 864ms/step


2024/05/07 05:59:47 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmplede3t28/model, flavor: tensorflow). Fall back to return ['tensorflow==2.15.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
2024/05/07 05:59:50 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under mlflow-artifacts:/547817128369496650/adf8496741a04249b7d6b0e5d06f5df9/artifacts. Set logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)` to see the full traceback.


RestException: INVALID_PARAMETER_VALUE: The run adf8496741a04249b7d6b0e5d06f5df9 must be in 'active' lifecycle_stage.

# 2. MLflow를 활용한 모델 최적화 Tracking
* 직접 parameter 변경해가며 모델 최적화
* GridSearch 혹은 RandomSearch를 활용하여 모델 최적화